In [21]:
import pandas_datareader.data as web
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np
import warnings
import pandas as pd
from pandas import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
register_matplotlib_converters()
from time import time
warnings.filterwarnings("ignore")
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import pacf
from datetime import date
from datetime import datetime
import yfinance as yf
import plotly.graph_objects as go
from random import *

In [22]:
#Choose Trading Platform
broker_auswahl = pd.read_csv('Trading_Broker.csv')

broker = str(input("Gewünschter Online-Broker: "))
print(broker)

i = broker_auswahl[broker_auswahl['Anbieter'] == broker].index
print(i[0])

print(broker_auswahl.at[i[0], "Gebühr"])
order_fee = broker_auswahl.at[i[0], "Gebühr"]
min_ = broker_auswahl.at[i[0], "Minimum"]
max_= broker_auswahl.at[i[0], "Maximum"]

IG
1
0.05


In [23]:
#Funktion zur Berechnung der Trading-Gebühren
def trading_fee(endowment):
    fee = endowment*(order_fee/100)
    
    #Prüfung, ob Mindest- oder Maximalgebühr des ausgewählten Brokers greift
    if fee <= min_:
        fee = min_
        
    if max_.isnumeric():
        if fee >= max_:
            fee = max_
        
    return fee

In [24]:
#Nutzerentscheidung, ob Short-Selling aktiviert werden soll -> Eingabe: "Yes" oder "No"
short = False
answer = str(input("Include short-selling?: "))
print(answer)
if answer == "Yes":
    short = True

Yes


In [25]:
#Funktion zum Download der relevanten Daten
def get_data(asset, start, end):
    ticker = asset
    data = yf.download(ticker, start=start, end=end)

    # calculate returns and format df
    data["Returns"] = 100 * data["Adj Close"].pct_change()
    data = data.dropna()
    data["Date"] = pd.to_datetime(data.index)

    data.reset_index(drop=True,inplace=True)
    return data

In [26]:
#In dieser Funktion findet das eigentliche Trading und die zugehörigen Berechnungen statt
def trade(data, trade_day_index, endowment, profit, cash):
    action = []
    real_returns = []
    forecasts = []
    tax_allowance = 801

    # forecast ist der Kurs an den zwei Vortagen
    forecast = data["Returns"][trade_day_index - 1] + data["Returns"][trade_day_index - 2]
    real_return = data["Returns"][trade_day_index]
    
    prev_endowment = endowment
        
    
    if forecast >= 0.0 and cash == True:  #BUY
        #TODO: Anpassung der BetSize
        endowment = endowment*(real_return/100+1) - trading_fee(endowment) # Berücksichtigung von trading fees
        action.append("BUY")
        cash = False

    elif forecast >= 0.0 and cash == False:  #HOLD
        endowment = endowment*(real_return/100+1)
        action.append("HOLD")
        cash = False

    elif forecast <= 0.0 and cash == True:
        if short == True: #short-selling
            endowment = endowment - (real_return/100)*endowment - trading_fee(endowment) # Berücksichtigung von trading fees
            gain = endowment - prev_endowment
            if isinstance(gain, float) == False:
                gain = gain.item()
            if gain > 0.0:
                profit = profit + gain
                if profit < 801:
                    endowment = endowment + gain
                else:
                    endowment = endowment + (1-0.26375)*gain
            action.append("SHORT")
            cash == True
        else: #HOLD
            endowment = endowment
            action.append("HOLD")
            cash = True

    elif forecast <= 0.0 and cash == False:  #SELL
        endowment = endowment - trading_fee(endowment) # Berücksichtigung von trading fees
        gain = endowment - prev_endowment
        if isinstance(gain, float) == False: #die isinstance-Abfragen (auch die weiter unten) werden nur gemacht weil es bei den folgenden if-Abfragen sonst zu Fehlermeldungen wegen dem zugehörigen Index kommt
            gain = gain.item()
            #Berücksichtigung steuerlicher Freibetrag, falls drüber, Abzug Kapitalertragssteuer + Kirchensteuer vom Gewinn aus Trade
        if gain >= 0:
            profit = profit + gain
            #Falls der Gewinn 801 GE noch nicht übersteigt, wird der volle Gewinn aus diesem Trade angerechnet. Andernfalls werden die Kapitalertragssteuern + Soli (und ggf. Kirchensteuer) abgezogen
            if profit < 801:
                endowment = endowment + gain
            else:
                endowment = endowment + (1-0.26375)*gain
        action.append("SELL")
        #prev_endowment = endowment
        cash = True
        
    if isinstance(endowment, float) == False:
        endowment = endowment.item()
    
    
    return data, endowment, action, profit, cash

In [27]:
today = date.today()
#profit = 0 #evtl. hier richtig
#stocks = ["DTG.DE","ROTH.PA", "DBK.DE", "WMT", "BA", "BRK-B", "CMG", "NFC.F", "MSFT", "WBD", "JNJ", "BP", "DIS", "BAC", "VOLV-B.ST", "AMZN","AAPL","BMW.DE", "SBUX", "GS", "RHM.DE", "AHLA.DE", "BEI.DE", "CCC3.F", "BAYN.DE", "BOSCHLTD.BO", "LHA.DE", "HLT", "TUI1.DE", "MCD", "MC.PA", "GOOG", "MSCI", "MAR", "GE", "TSLA", "LUV", "ATVI", "V", "RWE.DE"]
stocks = ["ROTH.PA", "DBK.DE", "WMT", "BA", "BRK-B", "CMG", "NFC.F", "MSFT", "WBD", "JNJ", "BP", "DIS", "BAC", "AMZN","AAPL","BMW.DE", "SBUX", "GS", "RHM.DE", "AHLA.DE", "BEI.DE", "CCC3.F", "BAYN.DE", "BOSCHLTD.BO", "LHA.DE", "HLT", "TUI1.DE", "MCD", "MC.PA", "GOOG", "MSCI", "MAR", "GE", "TSLA", "LUV", "ATVI", "V", "RWE.DE"]
#stocks = ["AAPL"]
endowment_after_trade = {}
ergebnis = []
number_stocks = len(stocks)
counter = 0
all_actions = {} # Kaufenscheidungen pro Aktie pro Tag

for stock in stocks:
    profit = 0
    start = datetime(2020, 12, 29) # Erster Prognose Tage
    start = start.date()
    trade_day = datetime(2021, 1, 4) # Erster Handelstag
    trade_day = trade_day.date()
    trade_day_index = 2
    end = datetime(2021, 4, 1) # Letzter Handelstag
    end = end.date()
    print(stock)
    endowment = 10000.0 #Wie viel Geld soll in jeweilige Aktie investiert werden
    start_endowment = endowment
    cash = True

    actions = []

    while trade_day != end: #Eventuell fehlermeldung weil in end zeit mit drin ist
    
        print(trade_day)
    
        if np.is_busday(trade_day) == True:
            data = get_data(stock, start, end+timedelta(days=5))            
            data, endowment, action, profit, cash = trade(data, trade_day_index, endowment, profit, cash)
            actions.append(action)
            print(action)
            trade_day_index += 1
            
            #if endowment <= start_endowment*0.96:
                #break
            
        else:
            print("No trading day (Weekend etc.)")
    
        
        trade_day = trade_day + timedelta(days=1)
        print(endowment)
        
    
    print(endowment)
    ergebnis.append(endowment)
    endowment_after_trade[stock]=endowment
    all_actions[stock] = actions
    


print(endowment_after_trade)
print(all_actions)
print(ergebnis)



AAPL
2021-01-04
[*********************100%***********************]  1 of 1 completed
['SHORT']
10484.389444397828
2021-01-05
[*********************100%***********************]  1 of 1 completed
['SHORT']
10349.520027542418
2021-01-06
[*********************100%***********************]  1 of 1 completed
['SHORT']
11035.933583579405
2021-01-07
[*********************100%***********************]  1 of 1 completed
['SHORT']
10653.832912337133
2021-01-08
[*********************100%***********************]  1 of 1 completed
['BUY']
10740.461330142572
2021-01-09
No trading day (Weekend etc.)
10740.461330142572
2021-01-10
No trading day (Weekend etc.)
10740.461330142572
2021-01-11
[*********************100%***********************]  1 of 1 completed
['HOLD']
10490.75752609447
2021-01-12
[*********************100%***********************]  1 of 1 completed
['SELL']
10485.512147331423
2021-01-13
[*********************100%***********************]  1 of 1 completed
['SHORT']
10310.125308190498
2021-01-